<a href="https://colab.research.google.com/github/Romulomrs/MortesPalestina/blob/main/CleaningData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_json('dados.json')

In [25]:
pd.set_option('display.width', 1000)  # Aumenta o limite horizontal
pd.set_option('display.max_columns', None)  # Permite mostrar todas as colunas

In [26]:
# Preenche valores ausentes somente nas colunas numéricas
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(0)

In [27]:
df.fillna(0, inplace = True)
df = df.drop(['ext_massacres_cum','ext_killed','ext_killed_cum',
                 'ext_killed_children_cum','ext_killed_women_cum','ext_injured','ext_injured_cum','ext_civdef_killed_cum',
                 'ext_med_killed_cum','ext_press_killed_cum',],axis = 1)


In [28]:
# Converte a data para datetime
df['report_date'] = pd.to_datetime(df['report_date'])

In [29]:
# Cria coluna: dias desde o início da guerra
inicio_guerra = pd.to_datetime('2023-10-06')
df['dias_desde_inicio'] = (df['report_date'] - inicio_guerra).dt.days

In [45]:
colunas = [
    'killed_cum',
    'injured_cum',
    'med_killed_cum',
    'press_killed_cum',
    'killed_children_cum',
    'killed_women_cum',
    'civdef_killed_cum'

]

# Substitui 0 por NA e faz forward fill em cada coluna
df[colunas] = df[colunas].replace(0, pd.NA).ffill()

In [46]:
colunas_categorias = [
    'Killed_cum',
    'killed_women_cum',
    'med_killed_cum',
    'press_killed_cum',
    'killed_children_cum',
    'civdef_killed_cum',
    'injured_cum',
]

In [47]:
# Filtra apenas os dados a partir de 7 de outubro de 2023
df = df[df['report_date'] >= inicio_guerra]


In [48]:
df_renomeado = df.rename(columns={
    'killed_cum': 'Geral',
    'killed_women_cum': 'Mulheres',
    'med_killed_cum': 'Médicos',
    'press_killed_cum': 'Imprensa',
    'killed_children_cum': 'Crianças',
    'civdef_killed_cum': 'DefesaCivil'
})

In [49]:
colunas_a_derreter = ['Geral', 'Mulheres', 'Médicos', 'Imprensa', 'Crianças', 'DefesaCivil']
id_vars = [col for col in df_renomeado.columns if col not in colunas_a_derreter]

# Agora sim, faz o melt mantendo tudo
df_long = pd.melt(
    df_renomeado,
    id_vars=id_vars,
    value_vars=colunas_a_derreter,
    var_name='Categoria',
    value_name='Mortos'
)

In [50]:
df_long.to_csv('dados.csv')

In [ ]:
print(df_long.columns)